In [7]:
import tensorflow_hub as hub
import tensorflow_text as text
import numpy as np
import tensorflow as tf
import pandas as pd
from senticnet.senticnet import SenticNet
import re
import string
from tensorflow.keras import datasets, layers, models



In [8]:
preprocess_url = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"
encoder_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4"

In [9]:
bert_preprocess_model = hub.KerasLayer(preprocess_url)
bert_model = hub.KerasLayer(encoder_url)

OSError: SavedModel file does not exist at: C:\Users\1027d\AppData\Local\Temp\tfhub_modules\602d30248ff7929470db09f7385fc895e9ceb4c0\{saved_model.pbtxt|saved_model.pb}

In [4]:
def bert_process(text_to_process):
    text_preprocessed = bert_preprocess_model(text_to_process)
    bert_results = bert_model(text_preprocessed)
    tempres = (bert_results["sequence_output"].numpy()).tolist()

    return tempres


In [5]:
def senticnet(processedinput, sentences):
    sn = SenticNet()
    index = 0
    # res = []
    arrmodel = [float(0)] * 20

    for i in range(len(processedinput)):
        processedinput[i][0].extend(arrmodel)
        processedinput[i][-1].extend(arrmodel)
        for j in range(1,len(processedinput[i]) -1):
            temparr = arrmodel
            try:
                cur_polarity_value = sn.polarity_value(sentences[index])
                Eclass = int((float(cur_polarity_value) + 1)/0.1)
                temparr[Eclass] = 1
                processedinput[i][j].extend(temparr)
            except:
                processedinput[i][j].extend(temparr)

            index += 1

    return processedinput


In [8]:
def clean_text(text):
    regex = re.compile('[%s]' % re.escape('|'))
    text = regex.sub(" ", text)
    words = str(text).split()
    words = [i.lower() + " " for i in words]
    words = [i for i in words if not "http" in i]
    words = " ".join(words)
    words = words.translate(words.maketrans('', '', string.punctuation))
    return words

In [9]:
def list_chunks(l, n):
    for i in range(0, len(l), n):
        yield l[i:i + n]

In [121]:
MBTI = pd.read_csv('mbti_1.csv')
data = []



In [122]:
for n in range(0,2):
    for post in MBTI["posts"][100*(n-1):(100 * n)]:
        sentences = []
        sentence = clean_text(str(post))
        sentence = sentence.split()
        if len(sentence) < 756:
            padd = ['0'] * (756-len(sentence))
            sentence.extend(padd)
        sentence = sentence[:756]
        tempsentence = list(list_chunks(sentence, 126))
        for wds in tempsentence:
            sentences.append(' '.join(wds))
        processedinput = bert_process(sentences)
        tokenizedoutput = senticnet(processedinput, sentence)
        data.append(tokenizedoutput)



In [125]:
# print(len(data))

500


In [129]:
# labels = []
IElabels = []
NSlabels = []
FTlabels = []
JPlabels = []

templabels = MBTI["type"]
# labelmap = {"INTJ" : 0.0, "INTP": 1.0, "ENTJ": 2.0, "ENTP" : 3.0, "INFJ": 4.0, "INFP": 5.0, "ENFJ": 6.0, "ENFP": 7.0, "ISTJ": 8.0, "ISFJ": 9.0, "ESTJ": 10.0, "ESFJ": 11.0, "ISTP": 12.0, "ISFP": 13.0, "ESTP": 14.0, "ESFP": 15.0}
# labeindex = ["INTJ", "INTP", "ENTJ", "ENTP", "INFJ", "INFP", "ENFJ", "ENFP", "ISTJ", "ISFJ", "ESTJ", "ESFJ", "ISTP", "ISFP", "ESTP", "ESFP"]
labelmap = {"I" : 0.0, "E": 1.0, "N": 2.0, "S" : 3.0, "F": 4.0, "T": 5.0, "J": 6.0, "P": 7.0}

for l in templabels:
    IElabels.append(labelmap[l[0]])
    NSlabels.append(labelmap[l[1]])
    FTlabels.append(labelmap[l[2]])
    JPlabels.append(labelmap[l[3]])


In [131]:
traindata = data[:190]
testdata = data[190:200]
trainlabel = IElabels[:190]
testlabel = IElabels[190:200]


In [132]:
# traindata = data
# trainlabel = labels[200:700]

In [133]:
# print(len(testdata))

In [134]:
# print(len(traindata))

# print(len(trainlabel))

In [136]:
traindata = tf.convert_to_tensor(traindata)
testdata = tf.convert_to_tensor(testdata)
trainlabel = tf.convert_to_tensor(trainlabel)
testlabel = tf.convert_to_tensor(testlabel)

In [137]:
# import torch
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# import torchvision
# import torchvision.transforms as transforms
# import matplotlib.pyplot as plt
# import numpy as np
# import torch.optim as optim


In [138]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [139]:
# num_epochs = 5
# batch_size = 20
# learning_rate = 0.001

In [140]:
# class ConvNet(nn.Module):
#     def __init__(self):
#         super(ConvNet, self).__init__()
#         self.conv1 = nn.Conv2d(6, 5, (1,3))
#         self.pool = nn.MaxPool2d(2, 2)
#         self.conv2 = nn.Conv2d(5, 4, 3)
#         self.pool = nn.MaxPool2d(2, 2)
#         self.conv3 = nn.Conv2d(4, 3, (3,1))

#         self.fc1 = nn.Linear(6 * 3 * 3, 120)
#         self.fc2 = nn.Linear(120, 84)
#         self.fc3 = nn.Linear(84, 10)

#     def forward(self, x):
#         # -> n, 3, 32, 32
#         x = nn.LeakyReLU(self.conv1(x)) # -> n, 6, 14, 14
#         x = nn.LeakyReLU(self.conv2(x))  # -> n, 16, 5, 5
#         print(x.shape())
#         x = x.view(-1, 16 * 5 * 5)            # -> n, 400
#         x = F.relu(self.fc1(x))               # -> n, 120
#         x = F.relu(self.fc2(x))               # -> n, 84
#         x = self.fc3(x)                       # -> n, 10
#         return x

In [141]:
# model = ConvNet().to(device)

In [142]:
# criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)


In [143]:
# for epoch in range(num_epochs):  # loop over the dataset multiple times

#     running_loss = 0.0
#     for i, data in enumerate(traindata, 0):
#         # get the inputs; data is a list of [inputs, labels]
#         inputs = data
#         labels = trainlabel[i]
#         # zero the parameter gradients
#         optimizer.zero_grad()

#         # forward + backward + optimize
#         outputs = model(inputs)
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()

#         # print statistics
#         running_loss += loss.item()
#         if i % 2000 == 1999:    # print every 2000 mini-batches
#             print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
#             running_loss = 0.0

In [144]:
# PATH = './cnn.pth'
# torch.save(model.state_dict(), PATH)

In [145]:
# with torch.no_grad():
#     n_correct = 0
#     n_samples = 0
#     n_class_correct = [0 for i in range(10)]
#     n_class_samples = [0 for i in range(10)]
#     for images, labels in test_loader:
#         images = images.to(device)
#         labels = labels.to(device)
#         outputs = model(images)
#         # max returns (value ,index)
#         _, predicted = torch.max(outputs, 1)
#         n_samples += labels.size(0)
#         n_correct += (predicted == labels).sum().item()
        
#         for i in range(batch_size):
#             label = labels[i]
#             pred = predicted[i]
#             if (label == pred):
#                 n_class_correct[label] += 1
#             n_class_samples[label] += 1

#     acc = 100.0 * n_correct / n_samples
#     print(f'Accuracy of the network: {acc} %')

#     for i in range(10):
#         acc = 100.0 * n_class_correct[i] / n_class_samples[i]
#         print(f'Accuracy of {classes[i]}: {acc} %')

In [146]:
# len(testlabel)

In [147]:
# print(testlabel)

In [148]:
# print(type(traindata))
# print(type(trainlabel))

In [149]:
iemodel = models.Sequential()
iemodel.add(layers.Conv2D(32, (1, 3), activation='relu', input_shape=traindata.shape[1:]))
# model.add(layers.MaxPooling2D((1, 1)))
iemodel.add(layers.Conv2D(32, (3, 3), activation='relu'))
# model.add(layers.MaxPooling2D((1, 1), padding='same'))
iemodel.add(layers.Conv2D(32, (3, 1), activation='relu'))

In [150]:
# model = tf.keras.models.load_model('my_model.h5')

In [151]:
iemodel.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 6, 126, 32)        75680     
                                                                 
 conv2d_4 (Conv2D)           (None, 4, 124, 32)        9248      
                                                                 
 conv2d_5 (Conv2D)           (None, 2, 124, 32)        3104      
                                                                 
 flatten_1 (Flatten)         (None, 7936)              0         
                                                                 
 dense_1 (Dense)             (None, 128)               1015936   
                                                                 
Total params: 1,103,968
Trainable params: 1,103,968
Non-trainable params: 0
_________________________________________________________________


In [152]:
iemodel.add(layers.Flatten())
iemodel.add(layers.Dense(128, activation='relu'))


In [153]:
iemodel.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 6, 126, 32)        75680     
                                                                 
 conv2d_4 (Conv2D)           (None, 4, 124, 32)        9248      
                                                                 
 conv2d_5 (Conv2D)           (None, 2, 124, 32)        3104      
                                                                 
 flatten_1 (Flatten)         (None, 7936)              0         
                                                                 
 dense_1 (Dense)             (None, 128)               1015936   
                                                                 
 flatten_3 (Flatten)         (None, 128)               0         
                                                                 
 dense_3 (Dense)             (None, 128)              

In [154]:
iemodel.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [155]:
history = iemodel.fit(traindata, trainlabel, epochs=5, 
                    validation_data=(testdata, testlabel))

In [156]:
# history = iemodel.fit(traindata, trainlabel, epochs=8)

Epoch 1/8


In [ ]:
iemodel.save("my_iemodel.h5")

In [ ]:
# post = MBTI["posts"][1505]
# sentences = []
# sentence = clean_text(str(post))
# sentence = sentence.split()
# if len(sentence) < 756:
#     padd = ['0'] * (756-len(sentence))
#     sentence.extend(padd)
# sentence = sentence[:756]
# tempsentence = list(list_chunks(sentence, 126))
# for wds in tempsentence:
#     sentences.append(' '.join(wds))
# processedinput = bert_process(sentences)
# tokenizedoutput = senticnet(processedinput, sentence)
# data.append(tokenizedoutput)

In [ ]:
# sample = [data[-1]]
# sample = tf.convert_to_tensor(sample)

# samplea = [labelmap[MBTI["type"][1505]]]
# samplea = tf.convert_to_tensor(samplea)

In [ ]:
# results = model.evaluate(sample, samplea)

1/1 [==============================] - 0s 164ms/step - loss: 2.6632 - accuracy: 0.0000e+00


In [ ]:
# result = model.predict(sample)
# print(labelmap[MBTI["type"][1505]])
# print(result)

1/1 [==============================] - 0s 180ms/step
4.0
[[7.5876746 7.1726966 4.733189  9.027504  6.875262  6.8005047 4.7541246
  3.2605245 0.        3.5965374 0.        0.        0.        0.
  0.        0.        0.        0.        0.        0.        0.
  0.        0.        0.        0.        0.        0.        0.
  0.        0.        0.        0.        0.        0.        0.
  0.        0.        0.        0.        0.        0.        0.
  0.        0.        0.        0.        0.        0.        0.
  0.        0.        0.        0.        0.        0.        0.
  0.        0.        0.        0.        0.        0.        0.
  0.        0.        0.        0.        0.        0.        0.
  0.        0.        0.        0.        0.        0.        0.
  0.        0.        0.        0.        0.        0.        0.
  0.        0.        0.        0.        0.        0.        0.
  0.        0.        0.        0.        0.        0.        0.
  0.        0.        0.  